In [1]:
import keras
import numpy as np
from keras import backend as k
from keras import optimizers
from keras.layers import *
from keras.models import Model
from keras.callbacks import TensorBoard, ModelCheckpoint
from keras.models import load_model
from keras.engine.topology import Layer
from keras.datasets import mnist
from keras.losses import binary_crossentropy
from keras_tqdm import TQDMNotebookCallback
import pdb

Using TensorFlow backend.


In [2]:
batchsize = 2
n_z = 20
m = 64

In [ ]:
#def loss():
    # p_z = gaussian(0,1)
    # q_z_x = gaussian(mean_y, e^ln_var_y) - check this exp
    # p_x_z = gaissian(mean_x, e^ln_var_x)
    # L = F.gaussian_kl_dvergance(q_z_x, p_z) - F.gaussian_nll(p_x_z)
        
    #? What about input x?
        
    # Why use log variance instead of variance:

In [3]:
def sample_z(args):
    mu, log_sigma = args
    batch = K.shape(mu)[0]
    dim = K.int_shape(mu)[1]
    eps = K.random_normal(shape=(batch, dim), mean=0., stddev=1.)
    return mu + K.exp(log_sigma / 2) * eps

In [4]:
(x_train, y_train), (x_test, y_test) = mnist.load_data()
batches = int(len(x_train)/batchsize)

In [5]:
image_size = x_train.shape[1]
x_train = np.reshape(x_train, [-1, image_size, image_size, 1])
x_test = np.reshape(x_test, [-1, image_size, image_size, 1])
x_train = x_train.astype('float32') / 255
x_test = x_test.astype('float32') / 255

In [6]:
x_train1 = x_train[:10]

In [7]:
def batch_generator():
    
    while True:

        for batch in range(batches):
            x = []
            
            data = x_train[batch * batchsize: (batch + 1) * batchsize]
            for i in data:
                x.append(np.reshape(i, (28, 28, 1)))
            
            x_train1 = np.asarray(x)
            yield (x_train1, x_train1) # the data is also the target (since we're re-generating images)

In [8]:
inputs1 = Input(shape = (28, 28, 1))

encode = Conv2D(32, (2, 2), strides=2, padding = 'same')(inputs1)

encode = Activation('relu')(encode)

encode = Conv2D(32, (2, 2), strides=2, padding = 'same')(encode)

encode = Activation('relu')(encode)

out_shape = list(k.int_shape(encode))

encode = Flatten()(encode)

encode = Dense(512, activation = 'relu')(encode)

encode = Dropout(0.5)(encode)

mu = Dense(n_z, activation = 'linear')(encode)

log_sigma = Dense(n_z, activation = 'linear')(encode)

latent_vector = Lambda(sample_z, output_shape=(n_z,))([mu, log_sigma])

model_encoder = Model(inputs = inputs1, outputs = [mu, log_sigma, latent_vector])

In [9]:
model_encoder.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 28, 28, 1)    0                                            
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 14, 14, 32)   160         input_1[0][0]                    
__________________________________________________________________________________________________
activation_1 (Activation)       (None, 14, 14, 32)   0           conv2d_1[0][0]                   
__________________________________________________________________________________________________
conv2d_2 (Conv2D)               (None, 7, 7, 32)     4128        activation_1[0][0]               
__________________________________________________________________________________________________
activation

In [10]:
inputs2 = Input(shape = (n_z,))

decode = Dense(out_shape[1] * out_shape[2] * out_shape[3], activation = 'relu')(inputs2)

decode = Reshape((out_shape[1], out_shape[2], out_shape[3]))(decode)

decode = Conv2DTranspose(32, (2, 2), strides=2, padding = 'same')(decode)

decode = Activation('relu')(decode)

decode = Conv2DTranspose(32, (2, 2), strides=2, padding = 'same')(decode)

decode = Activation('relu')(decode)

out = Conv2DTranspose(1, (2, 2), activation = 'sigmoid', padding = 'same')(decode)

model_decoder = Model(inputs = inputs2, outputs = out)

In [11]:
model_decoder.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, 20)                0         
_________________________________________________________________
dense_4 (Dense)              (None, 1568)              32928     
_________________________________________________________________
reshape_1 (Reshape)          (None, 7, 7, 32)          0         
_________________________________________________________________
conv2d_transpose_1 (Conv2DTr (None, 14, 14, 32)        4128      
_________________________________________________________________
activation_3 (Activation)    (None, 14, 14, 32)        0         
_________________________________________________________________
conv2d_transpose_2 (Conv2DTr (None, 28, 28, 32)        4128      
_________________________________________________________________
activation_4 (Activation)    (None, 28, 28, 32)        0         
__________

In [12]:
outputs = model_decoder(model_encoder(inputs1)[2])
model = Model(inputs1, outputs)

In [13]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 28, 28, 1)         0         
_________________________________________________________________
model_1 (Model)              [(None, 20), (None, 20),  828136    
_________________________________________________________________
model_2 (Model)              (None, 28, 28, 1)         41313     
Total params: 869,449
Trainable params: 869,449
Non-trainable params: 0
_________________________________________________________________


In [14]:
reconstruction_loss = binary_crossentropy(K.flatten(inputs1), K.flatten(outputs))

In [15]:
reconstruction_loss *= image_size * image_size

In [16]:
kl_loss = 1 + log_sigma - K.square(mu) - K.exp(log_sigma)
kl_loss = K.sum(kl_loss, axis=-1)
kl_loss *= -0.5

In [17]:
loss_ = K.mean(reconstruction_loss + kl_loss)

In [18]:
model.add_loss(loss_)

In [19]:
adam_ = optimizers.Adam(lr = 1e-3, beta_1=0.9, beta_2=0.999, epsilon=1e-8)

In [20]:
model.compile(optimizer = adam_)

In [ ]:
model.fit(x_train1, batch_size = batches, epochs=10, shuffle = True, verbose=0, callbacks=[TQDMNotebookCallback()])